# Advanced Medical Text Extraction

Here we will work to use mutiple adnvanced NLP models to validate and test how much advanced medical data such as the level and status of a particular ailment can be extracted. We wil start with examinations around NMIBC Grade - and see if we can identify the grade based on the medical motes int he text -

The Goal here being to unearth the best model or combination of models to be used for further NLP training around that target data down the line

<br/>

Reaserch Links -
https://colab.research.google.com/drive/1uqc5urkMetu_6J1vU7bTYC6t4JsQ5Arl#scrollTo=ncrDHyf7vPp_

https://colab.research.google.com/drive/1pW0hOXz6OTB7bxRTuaM8rnWMM2mGTOFh


https://colab.research.google.com/drive/1z42H6J_7qGEnNdBodGsZ1G3-xtnoGRxz

## BERT NER
First lets see what we are able to extract directly with BERT NER -

bert-base-NER is a fine-tuned BERT model that is ready to use for Named Entity Recognition and achieves state-of-the-art performance for the NER task. It has been trained to recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).Jul 8, 2022

### Gather installs and Imports needed

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

### Set up the mock example text data

In [3]:
Text = "The patient came in today with PSA-150 on 10/29/1999. This is indicative of cancer. Unlike on 10/2/1999 when PSA was 2, not cancer indicative."

### Pull in the model

In [4]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Use the model to extract NER -

In [5]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(Text)
print(' Full results')
print(ner_results)
print()
print('Results Break down')

for x in ner_results:
  print(x)

 Full results
[{'entity': 'B-MISC', 'score': 0.9599354, 'index': 7, 'word': 'PS', 'start': 31, 'end': 33}, {'entity': 'I-MISC', 'score': 0.9627516, 'index': 8, 'word': '##A', 'start': 33, 'end': 34}, {'entity': 'B-MISC', 'score': 0.6849659, 'index': 33, 'word': 'PS', 'start': 109, 'end': 111}, {'entity': 'I-MISC', 'score': 0.8178611, 'index': 34, 'word': '##A', 'start': 111, 'end': 112}]

Results Break down
{'entity': 'B-MISC', 'score': 0.9599354, 'index': 7, 'word': 'PS', 'start': 31, 'end': 33}
{'entity': 'I-MISC', 'score': 0.9627516, 'index': 8, 'word': '##A', 'start': 33, 'end': 34}
{'entity': 'B-MISC', 'score': 0.6849659, 'index': 33, 'word': 'PS', 'start': 109, 'end': 111}
{'entity': 'I-MISC', 'score': 0.8178611, 'index': 34, 'word': '##A', 'start': 111, 'end': 112}


As we see here basic NER is more conserned with names than anything else and can't get much info for use around the medical Targets Themselves we will have to Dig Deeper -

#### BERT NER -Final Model Score - Non Viable For Use

## Using Key Bert
Next lets try a more advanced for of text extraction around any and all Key targets being disscussed if we can find these in our extraction we can pair it with regex to futher fillter results and vastly reduce the text we need to search for our targets



KeyBERT is a minimal and easy-to-use keyword extraction technique that leverages BERT embeddings to create keywords and keyphrases that are most similar to a document.

Although there are already many methods available for keyword generation (e.g., Rake, YAKE!, TF-IDF, etc.) I wanted to create a very basic, but powerful method for extracting keywords and keyphrases. This is where KeyBERT comes in! Which uses BERT-embeddings and simple cosine similarity to find the sub-phrases in a document that are the most similar to the document itself.



In [6]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.0 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23776 sha256=f0fa417818496a9536626dbfeba373956ffe80f5b3365ee7a97be5832546e75a
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=009db5a2107956fbb046e403306de6eb75b2ee3aba14a19dc7d3917ef361fd57
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


Now that we have Key Bert Set Up Lets Use it to Extract Targets from our Text

In [7]:
from keybert import KeyBERT
kw_model = KeyBERT()

keys = kw_model.extract_keywords(Text, keyphrase_ngram_range=(3, 3), top_n = 10, stop_words='english', use_mmr=True, diversity=0.5)
keys

[('psa cancer indicative', 0.7982),
 ('1999 psa cancer', 0.7779),
 ('1999 indicative cancer', 0.6692),
 ('today psa 150', 0.6071),
 ('10 1999 psa', 0.5529),
 ('indicative cancer unlike', 0.5129),
 ('came today psa', 0.5079),
 ('patient came today', 0.4141),
 ('cancer unlike 10', 0.3998),
 ('29 1999 indicative', 0.3815)]

Now This is much better we are seeing information pulled around Grade and level of Risk as well as the specific condition NMIBC but we have some issues.

* 1 we have a lot going on around more than just NMIBC grade
* 2 info on it seperated and we dont get the specifc call around grade 3
* 3 this simply classfys these sections of text as impotant in the sentence it dose not give us any other form of clasification

<br/>
Lets See if we can do better with a more medicaly targeted clasification based search


#### Key BERT -Final Model Score - Limited Viablity For Use

## Scispacy Medical Detection
Now we will use a model leveraging spacy based NLP central models but pre trained on ons of medical text to help further identify the targets of medical interest inside the actualcentral text


scispaCy is an open-source software library for advanced Natural Language Processing, written in the programming languages Python and Cython. The library is published under the MIT license and currently offers statistical neural network models for processing biomedical, scientific or clinical text.

In [8]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 829.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.8 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13547989 sha256=cd08d5e1ae3d8080fd70e3da6b973bb1f1138b608505c98c101edad7a5c9c28c
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully unin

Now import in the tools for Bio based NER that we need -

In [9]:
import scispacy
import spacy
nlp = spacy.load("en_ner_bc5cdr_md")

Now lets see what we can extract using it on our central targets


In [10]:
doc = nlp(Text)
print("TEXT", "START", "END", "ENTITY TYPE")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

TEXT START END ENTITY TYPE
10/29/1999 42 52 CHEMICAL
cancer 76 82 DISEASE
10/2/1999 94 103 CHEMICAL
cancer 124 130 DISEASE


Now thats more like it not only was the NBIM condition detected as primeary info we also got the whole Hig Grade state identified as a issuea as well it missed the grade three call but thats all it missed from what we are looking for and that is impressive With aaditonal traning this model could be exactly what we need to gather the data we seek -

#### Scispacy -Final Model Score - Decent Viability For Use

## Stanza Medical Detection
Next We Will try to a test with the model framework Stanza. Stanza is a Python NLP toolkit that supports 60+ human languages. It is built with highly accurate neural network components that enable efficient training and evaluation with your own annotated data, and offers pretrained models on 100 treebanks. Additionally, Stanza provides a stable, officially maintained Python interface to Java Stanford CoreNLP Toolkit.

Note that Stanza only supports Python 3.6 and above. Installing and importing Stanza are as simple as running the following commands:

In [11]:
!pip install Stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 21.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=92a52228d14e268d65a95c966f07d3a2f2b822a05fb2380975a2addc17db3fc8
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji


Next download the english model

In [12]:
# Import the package
import stanza

# Download an English model into the default directory
print("Downloading English model...")
stanza.download('en')

INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


## Processing Text
Constructing Pipeline To process a piece of text, you'll need to first construct a Pipeline with different Processor units. The pipeline is language-specific, so again you'll need to first specify the language (see examples).

In [13]:
# Build an English pipeline, with all processors by default
print("Building an English pipeline...")
en_nlp = stanza.Pipeline('en',verbose=True)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


Building an English pipeline...


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


## Annotating Text
After a pipeline is successfully constructed, you can get annotations of a piece of text simply by passing the string into the pipeline object. The pipeline will return a Document object, which can be used to access detailed annotations from. For example:

In [14]:
# Processing English text
en_doc = en_nlp(Text)
print(type(en_doc))

<class 'stanza.models.common.doc.Document'>


We can now have the model perform a full prediction on the structure and anotation of the text-

In [15]:
for i, sent in enumerate(en_doc.sentences):
    print("[Sentence {}]".format(i+1))

    for word in sent.words:
        print("{:12s}\t{:12s}\t{:6s}\t{:d}\t{:12s}".format(\
              word.text, word.lemma, word.pos, word.head, word.deprel))

    print("")

[Sentence 1]
The         	the         	DET   	2	det         
patient     	patient     	NOUN  	3	nsubj       
came        	come        	VERB  	0	root        
in          	in          	ADV   	3	advmod      
today       	today       	NOUN  	3	obl:tmod    
with        	with        	ADP   	7	case        
PSA         	PSA         	PROPN 	3	obl         
-           	-           	SYM   	9	case        
150         	150         	NUM   	7	nmod        
on          	on          	ADP   	11	case        
10/29/1999  	10/29/1999  	NUM   	3	obl         
.           	.           	PUNCT 	3	punct       

[Sentence 2]
This        	this        	PRON  	3	nsubj       
is          	be          	AUX   	3	cop         
indicative  	indicative  	ADJ   	0	root        
of          	of          	ADP   	5	case        
cancer      	cancer      	NOUN  	3	obl         
.           	.           	PUNCT 	3	punct       

[Sentence 3]
Unlike      	unlike      	ADP   	3	case        
on          	on          	ADP   	3	case       

This can help us see if the things we are looking for can be indentifited by their sent struct type directly here it looks like we may have to many targets for that so lets move on to more cognative based extraction

## Now lets perform Basic NER
Running the NERProcessor simply requires the TokenizeProcessor. After the pipeline is run, the Document will contain a list of Sentences, and the Sentences will contain lists of Tokens. Named entities can be accessed through Document or Sentence’s properties entities or ents. Alternatively, token-level NER tags can be accessed via the ner fields of Token.

Accessing Named Entities for Sentence and Document Here is an example of performing named entity recognition for a piece of text and accessing the named entities in the entire document:

In [16]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')

doc = nlp(Text)
print()
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!



entity: PSA-150	type: PRODUCT
entity: 10/29/1999	type: CARDINAL
entity: 10/2/1999	type: DATE
entity: PSA	type: ORG
entity: 2	type: CARDINAL


Interesting and Much Better than BERT we got names times and numbers as well as a tracking of NMIBC although it thinks its a ORG this is very good for pulling out key data unfortenetly it dose seem to be unable to detect much around grade besides the number 3 so for this task the viabiity is low but for getting more data around the grade it might be perfect.

### Stansa NER -Final Model Score - Low Viability For Use

## Stansa Medical Text Extraction
Now lets use Stanzas medical libs to define our text further

In [17]:
stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
nlp = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'})

doc = nlp(Text)

# print out the entities
print()
for ent in doc.entities:
    print(f'{ent.text}\t{ent.type}')

INFO:stanza:Downloading these customized packages for language: en (English)...
| Processor       | Package |
-----------------------------
| tokenize        | mimic   |
| pos             | mimic   |
| lemma           | mimic   |
| depparse        | mimic   |
| ner             | i2b2    |
| forward_charlm  | mimic   |
| pretrain        | mimic   |
| backward_charlm | mimic   |



INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | mimic   |
| pos       | mimic   |
| lemma     | mimic   |
| depparse  | mimic   |
| ner       | i2b2    |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!



cancer	PROBLEM
PSA	TEST
cancer	PROBLEM


Very nice even better than sci spacy not only did we identify NMIBC and High Grade Tumors as the problem but we also were able to ID some of the text around treatment this is huge we colud still be a bit more focused on grade but that can come with a little model tuning around our labeled examples and further rules based extraction. As a starting point for the central NLP model this shows some real promise.

### Stanza Medical NER -Final Model Score - Strong Viability For Use

# BioBert NER
now lets try using Bio Bert, BioBERT is a contextualized language representation model, based on BERT, a pre-trained model which is trained on different combinations of general & biomedical domain corpora. One major problem with domain problems is that you have domain texts which are only understood by domain experts

In [18]:
!pip install -q transformers
!pip install -q simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) 

Now lets import the Bio Bert model we need as well as anything else we would like to have to help with data trasformation or evaluation.

In [19]:
from simpletransformers.ner import NERModel
from transformers import AutoTokenizer
import pandas as pd
import logging

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the bio BERT pre-trained model.
#model = NERModel('bert', 'dmis-lab/biobert-v1.1',use_cuda=False ,labels=custom_labels, args=train_args)
model = NERModel('bert', 'dmis-lab/biobert-v1.1', use_cuda=True)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Using the Model (Running Inference)
Running the model to do some predictions/inference is as simple as calling model.predict(samples). **bold text**

In [20]:
samples = [Text]

predictions, _ = model.predict(samples)
for idx, sample in enumerate(samples):
  print('{}: '.format(idx))
  for word in predictions[idx]:
    print('{}'.format(word))

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0: 
{'The': 'I-LOC'}
{'patient': 'I-LOC'}
{'came': 'I-LOC'}
{'in': 'B-PER'}
{'today': 'I-LOC'}
{'with': 'I-LOC'}
{'PSA-150': 'I-LOC'}
{'on': 'I-LOC'}
{'10/29/1999.': 'I-LOC'}
{'This': 'B-MISC'}
{'is': 'B-ORG'}
{'indicative': 'B-ORG'}
{'of': 'I-LOC'}
{'cancer.': 'I-LOC'}
{'Unlike': 'I-LOC'}
{'on': 'I-LOC'}
{'10/2/1999': 'I-LOC'}
{'when': 'I-LOC'}
{'PSA': 'I-LOC'}
{'was': 'I-LOC'}
{'2,': 'I-LOC'}
{'not': 'I-LOC'}
{'cancer': 'I-LOC'}
{'indicative.': 'B-ORG'}


As we can see the model can id text but in its own format and without much specific significance given on onset to our targets a lot of training would be needed to get this model to work for our needs but it dose run well with GPU support so there are advanateges to using it if we have the time and labeled examples to do so.

### Bio Bert NER -Final Model Score - Low Viability For Use - Without Extensive Training

## BERT Question Answer
BERT is a Bidirectional Encoder Representations from Transformers. It is one of the most popular and widely used NLP models. BERT models can consider the full context of a word by looking at the words that come before and after it, which is particularly useful for understanding the intent behind the query asked. Because of its bidirectionality, it has a deeper sense of language context and flow and hence, is used in a lot of NLP tasks nowadays. More details about BERT in the article along with the code.

Transformers library has a lot of different BERT models. It is easy to find a task-specific model from this library and do our task. So, let’s get started but let’s first look at our dataset.

### Asking questions direct
CoQA is a Conversational Question Answering dataset released by Stanford NLP in 2019. It is a large-scale dataset for building Conversational Question Answering Systems. This dataset aims to measure the ability of machines to understand a text passage and answer a series of interconnected questions that appear in a conversation. The unique feature about this dataset is that each conversation is collected by pairing two crowd workers to chat about a passage in the form of questions and answers and hence, the questions are conversational. To understand the format of the JSON data, please refer to this link. We will be using the story, question, and answer from the JSON dataset to form our data frame.

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.4 MB/s eta 0:00:00


import needed libs


In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

But for question answering tasks, we can even use the already trained model and get decent results even when our text is from a completely different domain. To get decent results, we are using a BERT model which is fine-tuned on the SQuAD benchmark. For our task, we will use the BertForQuestionAnswering class from the transformers library.

In [4]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


From here lets build a function that can take in text tokenize it and send it to the model

In [5]:
def question_answer(question, text):

    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)

    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a

    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)

    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]

    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."

    print("\nPredicted answer:\n{}".format(answer.capitalize()))

Now lets put it all togther

In [7]:
Text = '''The patient came in today with PSA-150 on 10/29/1999. This is indicative of cancer. Unlike on 10/2/1999 when PSA was 2, not cancer indicative.'''

In [12]:
question = 'tell me about PSA'

question_answer(Text, question)


Predicted answer:
Indicative of cancer


In [13]:
question = 'give me all dates related to PSA and their values'

question_answer(Text, question)


Predicted answer:
Psa - 150 on 10 / 29 / 1999 . this is indicative of cancer


## And Thats The Game Folks
As you can see with a little creative text promting around our exact needs we get all the data on NMIBC grade from the text clearly identified from our sample and use it to create full structured data around our cilnical reaserch tagets.

This is huge and even though Bert Q and A is expenisive we can create a hybrid model with regex that only uses it to get the exact info on grade when any info on grade is detected intially thus saveing massive ammounts of processing power and increasing our speeds ten fold


## Bert Q and A -Final Model Score - Massive Viability For Use